In [1]:
import tensorflow as tf

In [2]:
batch_size = 100
learning_rate = 0.01
training_epochs = 10

In [3]:
tf.summary.FileWriterCache.clear()

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
with tf.name_scope('input'):
    x =  tf.placeholder(tf.float32, [None, 784], name="x-input")
    y_ =  tf.placeholder(tf.float32, [None, 10], name="y-input")

In [6]:
# Reseau dans couche caché les vecteurs vont des 784 inputs -> 10 outputs
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([784, 10]))
#Bias (à revoir)
with tf.name_scope("biases"):
    b = tf.Variable(tf.zeros([10]))

In [7]:
with tf.name_scope("softmax"):
    y = tf.nn.softmax(tf.matmul(x,W) + b)
#reduce mean: moyenne arithmetique
#reduce sum: somme sur la deuxième dimention (0 vertical/colone - 1 horizontal/ligne)
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
#equal: revoie un boolean si les deux valeur sont egale ou pas
#argmax: renvoie l'index de la valeur la plus haute sur la dimention definit
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    #cast: change le type d'un tensor, ici bool vers float32
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
with tf.name_scope('train'):
    train_op =  tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [11]:
# create a summary for our cost and accuracy
tf.summary.scalar("cost", cross_entropy)
tf.summary.scalar("accuracy", accuracy)
summary_op = tf.summary.merge_all()

In [12]:

with tf.Session() as sess:
   init = tf.global_variables_initializer()
   sess.run(init)
   writer = tf.summary.FileWriter('output', sess.graph)
   
   for epoch in range(training_epochs):
      batch_count = int(mnist.train.num_examples/batch_size)
      for i in range(batch_count):
         batch_x, batch_y = mnist.train.next_batch(batch_size)

         _, summary = sess.run([train_op, summary_op], feed_dict={x: batch_x, y_: batch_y})
         
         writer.add_summary(summary, epoch * batch_count + i)
      if epoch % 2 == 0:
         print("Epoch: ", epoch)
   print("Accuracy: ", accuracy.eval(feed_dict={x:mnist.test.images, y_: mnist.test.labels}))
   writer.close()
   print("done")


Epoch:  0
Epoch:  2
Epoch:  4
Epoch:  6
Epoch:  8
Accuracy:  0.9021
done
